# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止（連番前提を活かして**並べ替え無し**で所望順に並べます）

## 1) 問題

* `連番 id（1..N）を持つ Seat で、2人ごとに席（id）を入れ替える。人数が奇数なら最後の1人は据え置き。結果は id 昇順。`
* 入力 DF: `Seat(id: int, student: str)`
* 出力: `id, student`（id 昇順。奇数↔直後の偶数で入替／末尾の孤立奇数はそのまま）

## 2) 実装（指定シグネチャ厳守）

> 0始まりに直して **LSB を XOR** でトグル（`((id-1) ^ 1) + 1`）→「末尾が奇数」の1件だけ据え置き。
> 並べ替え禁止のため、**連番 RangeIndex** を用いて `reindex` で 1..N の順を直接作ります。

```python
import pandas as pd

def swap_seats(seat: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'student']
    """
    if seat.empty:
        return seat[['id', 'student']]

    s = seat[['id', 'student']].copy()
    max_id = int(s['id'].max())

    # 1↔2, 3↔4, ... を一発で表す（XOR）
    new_id = ((s['id'] - 1) ^ 1) + 1

    # 末尾が奇数なら、その1件だけ据え置き（id = max_id の行）
    if (max_id & 1) == 1:
        new_id = new_id.where(s['id'] != max_id, max_id)

    # 並べ替え無しで id 昇順を実現：連番インデックスに再配置
    out_ids = pd.RangeIndex(1, max_id + 1)
    # new_id をインデックスにして student を並べ替えず配置
    placed = pd.Series(s['student'].to_numpy(), index=new_id.to_numpy())
    out = pd.DataFrame({'id': out_ids, 'student': placed.reindex(out_ids).to_numpy()})

    return out

Analyze Complexity
Runtime 274 ms
Beats 67.29%
Memory 66.96 MB
Beats 56.16%

```

## 3) アルゴリズム説明

* 使用API/手法

  * ビット演算（XOR）でペア入替：`((id-1) ^ 1) + 1`
  * `Series.where` で末尾奇数 1 件だけ据え置き
  * `RangeIndex` と `reindex` で **ソート無し**に `id` 昇順を構成（連番前提を活用）
* **NULL / 重複 / 型**

  * 入力は PK 連番なので重複や欠損は想定せず、`new_id` も一意
  * `id` は整数前提（演算前に `max_id` を `int` 化）

## 4) 計算量（概算）

* 全処理はベクトル演算主体で **O(N)**、追加メモリは `new_id` と `placed` の **O(N)**

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Seat 入力] --> B[0始まりに変換して XOR]
  B --> C[末尾の孤立奇数を据え置き]
  C --> D["RangeIndex(1..N) を用意"]
  D --> E["reindex で 1..N の順に配置"]
  E --> F[出力 id, student]
```

いい数値です！さらに数 ms・数 MB を詰めるなら、**Pandas のオブジェクト生成を最小化して NumPy で配列直書き**に振るのが有効です。`Series.where` や `reindex` をやめ、**1 回のインデックス代入**で完成させます。

---

# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `連番 id（1..N）を持つ Seat で 2人ごとに id を入替。人数が奇数なら最後の1人は据え置き。結果は id 昇順。`
* 入力 DF: `Seat(id: int, student: str)`
* 出力: `id, student`（id 昇順）

## 2) 実装（指定シグネチャ厳守）

> **完全ベクトル化 & 追加オブジェクト最小化**
> `((id-1) ^ 1) + 1` で 1↔2 / 3↔4 … を一括変換し、**最後が奇数のときだけ**ガード。
> 出力は **NumPy 配列に直接配置**してから DataFrame を 1 度だけ作成します。

```python
import pandas as pd
import numpy as np

def swap_seats(seat: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'student']
    """
    if seat.empty:
        return seat[['id', 'student']]

    # 参照を NumPy に落として中間 Series/DataFrame を作らない
    ids = seat['id'].to_numpy()
    students = seat['student'].to_numpy()

    max_id = int(ids.max())
    n = max_id  # 連番 1..N 前提

    # 1↔2, 3↔4, ... を一括変換
    new_ids = ((ids - 1) ^ 1) + 1

    # 末尾が奇数なら、最後の 1 件だけ据え置き
    if (max_id & 1) == 1:
        # True は最後の行 1 件のみ（連番前提）
        keep_mask = (ids == max_id)
        # np.where は新配列を返すが、分岐コストを最小化
        new_ids = np.where(keep_mask, ids, new_ids)

    # 出力 student を 1 回の代入で配置（0-based に直す）
    out_students = np.empty(n, dtype=students.dtype)
    out_students[new_ids - 1] = students

    # id は 1..N をそのまま並べる（並べ替え不要）
    out = pd.DataFrame({
        'id': np.arange(1, n + 1, dtype=ids.dtype),
        'student': out_students
    })
    return out

Analyze Complexity
Runtime 255 ms
Beats 92.64%
Memory 66.56 MB
Beats 82.53%

```

### ここが効きます

* `reindex` や `Series` 化をやめ、**NumPy への 1 回代入**に集約（メモリと CPU を削減）
* 偶奇判定は **ビット演算**（`& 1` / XOR）で高速化
* 中間コピー（`copy()`/`drop`/`where` の連鎖）を排除

## 3) アルゴリズム説明

* 使用 API/手法: `numpy` 演算（XOR、`where`、インデクシング）、`DataFrame` 最終 1 回生成
* **NULL / 重複 / 型**

  * 問題前提が PK 連番のため重複・欠損は非想定（`new_ids` も一意）
  * `students` は object/string。型変換は不要

## 4) 計算量（概算）

* すべて **O(N)**（1 回の代入で完成）、追加メモリは `new_ids` と `out_students` の **O(N)**
  ※ 既存案より **中間オブジェクトが減る**ため、実計測で **数 %〜十数 %** の改善余地

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Seat 入力] --> B[NumPy 配列へ取り出し]
  B --> C["XOR で 1↔2, 3↔4"]
  C --> D[最後の孤立奇数のみ据え置き]
  D --> E[出力配列に一括配置]
  E --> F[DataFrame を1回だけ生成]
```

---

### さらに詰めたい場合（任意）

* `students` が大きな文字列の場合、`Categorical` 化してから戻すと一部ケースでメモリが下がることがあります（ただし変換コストとトレードオフ）。
* 競技環境で許されるなら、関数外で `np.arange(1, n+1)` の再利用（キャッシュ）も微改善に。

